In [31]:
%pip install selenium parsel

You should consider upgrading via the 'c:\Users\anfiniti\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  # Import Service untuk mendefinisikan path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# Tentukan path ke Chromedriver
chromedriver_path = './chromedriver.exe'  # Ubah sesuai lokasi file Anda

# Buat objek Service dengan path driver
service = Service(chromedriver_path)

# Inisialisasi webdriver
driver = webdriver.Chrome(service=service)

# Buka halaman web
driver.get("https://www.google.com/maps/place/ITSNU+Pekalongan/@-6.9606099,109.6361072,17z/data=!4m8!3m7!1s0x2e7021b4d2b766bb:0x40dacfe03dd23e15!8m2!3d-6.9606152!4d109.6386821!9m1!1b1!16s%2Fg%2F11fjn8_k61?entry=ttu&g_ep=EgoyMDI1MDEyMi4wIKXMDSoASAFQAw%3D%3D")

def scroll_to_load_reviews():
    for _ in range(5):  # Scroll 5 kali
        scrollable_div = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="m6QErb DxyBCb kA9KIf dS8AEf ecceSd"]'))
        )
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(3)  # Tunggu loading selesai

# Klik tombol "Lihat semua ulasan" jika ada
try:
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(text(),"Lihat semua ulasan")]'))
    ).click()
    time.sleep(3)
except:
    print("Tidak ada tombol untuk melihat semua ulasan.")

# Scroll untuk memuat lebih banyak ulasan
scroll_to_load_reviews()

# Fungsi untuk mengambil ulasan dan bintang
def get_reviews_and_stars():
    reviews = driver.find_elements(By.XPATH, '//div[@class="jftiEf fontBodyMedium"]')
    stars = driver.find_elements(By.XPATH, '//span[@class="kvMYJc"]')

    reviews_list = []
    for review, star in zip(reviews, stars):
        review_text = review.text
        star_rating = star.get_attribute("aria-label")
        reviews_list.append([review_text, star_rating])

    return reviews_list

# Ambil ulasan dan bintang
reviews_and_stars = get_reviews_and_stars()

# Simpan ke CSV
csv_file = "reviews.csv"
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Review", "Star Rating"])
    writer.writerows(reviews_and_stars)

print(f"Data telah disimpan ke {csv_file}")

# Tutup driver
driver.quit()

Tidak ada tombol untuk melihat semua ulasan.


TimeoutException: Message: 
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0035A813+48355]
	(No symbol) [0x002EC4B1]
	(No symbol) [0x001F5358]
	(No symbol) [0x002209A5]
	(No symbol) [0x00220B3B]
	(No symbol) [0x0024E232]
	(No symbol) [0x0023A784]
	(No symbol) [0x0024C922]
	(No symbol) [0x0023A536]
	(No symbol) [0x002182DC]
	(No symbol) [0x002193DD]
	GetHandleVerifier [0x005BAABD+2539405]
	GetHandleVerifier [0x005FA78F+2800735]
	GetHandleVerifier [0x005F456C+2775612]
	GetHandleVerifier [0x003E51E0+616112]
	(No symbol) [0x002F5F8C]
	(No symbol) [0x002F2328]
	(No symbol) [0x002F240B]
	(No symbol) [0x002E4FF7]
	BaseThreadInitThunk [0x75547BA9+25]
	RtlInitializeExceptionChain [0x773FC0CB+107]
	RtlClearBits [0x773FC04F+191]


Tidak ada tombol untuk melihat semua ulasan.


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="m6QErb DxyBCb kA9KIf dS8AEf ecceSd"]"}
  (Session info: chrome=114.0.5735.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0035A813+48355]
	(No symbol) [0x002EC4B1]
	(No symbol) [0x001F5358]
	(No symbol) [0x002209A5]
	(No symbol) [0x00220B3B]
	(No symbol) [0x0024E232]
	(No symbol) [0x0023A784]
	(No symbol) [0x0024C922]
	(No symbol) [0x0023A536]
	(No symbol) [0x002182DC]
	(No symbol) [0x002193DD]
	GetHandleVerifier [0x005BAABD+2539405]
	GetHandleVerifier [0x005FA78F+2800735]
	GetHandleVerifier [0x005F456C+2775612]
	GetHandleVerifier [0x003E51E0+616112]
	(No symbol) [0x002F5F8C]
	(No symbol) [0x002F2328]
	(No symbol) [0x002F240B]
	(No symbol) [0x002E4FF7]
	BaseThreadInitThunk [0x75547BA9+25]
	RtlInitializeExceptionChain [0x77